### Introduction

Here you'll learn to build models using Catboost, Lightgbm and NaiveBayes algorithm in Python. Given the text classification problem, you'll also learn to clean data, create bag of words matrix, tf-idf matrix. 

On top of what's done here, next you can create a simple voting ensemble from the predictions generated from these models here.

In [1]:
# Load Libraries
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

In [3]:
# load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [7]:
# function to clean data

stops = set(stopwords.words("english"))
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in txt.split()])

    return txt

In [9]:
## join data
test['Is_Response'] = np.nan
alldata = pd.concat([train, test]).reset_index(drop=True)

In [11]:
# clean description
alldata['Description'] = alldata['Description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))

In [13]:
# initialise the functions - we'll create separate models for each type.
countvec = CountVectorizer(analyzer='word', ngram_range = (1,1), min_df=150, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', ngram_range = (1,1), min_df = 150, max_features=500)

In [14]:
# create features
bagofwords = countvec.fit_transform(alldata['Description'])
tfidfdata = tfidfvec.fit_transform(alldata['Description'])

In [15]:
# label encode categorical features in data given
cols = ['Browser_Used','Device_Used']

for x in cols:
    lbl = LabelEncoder()
    alldata[x] = lbl.fit_transform(alldata[x])

In [16]:
# create dataframe for features
bow_df = pd.DataFrame(bagofwords.todense())
tfidf_df = pd.DataFrame(tfidfdata.todense())

In [17]:
# set column names
bow_df.columns = ['col'+ str(x) for x in bow_df.columns]
tfidf_df.columns = ['col' + str(x) for x in tfidf_df.columns]

In [18]:
# create separate data frame for bag of words and tf-idf

bow_df_train = bow_df[:len(train)]
bow_df_test = bow_df[len(train):]

tfid_df_train = tfidf_df[:len(train)]
tfid_df_test = tfidf_df[len(train):]

In [219]:
# split the merged data file into train and test respectively
train_feats = alldata[~pd.isnull(alldata.Is_Response)]
test_feats = alldata[pd.isnull(alldata.Is_Response)]

In [220]:
### set target variable

train_feats['Is_Response'] = [1 if x == 'happy' else 0 for x in train_feats['Is_Response']]

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [24]:
# merge count (bag of word) features into train
train_feats1 = pd.concat([train_feats[cols], bow_df_train], axis = 1)
test_feats1 = pd.concat([test_feats[cols], bow_df_test], axis=1)

test_feats1.reset_index(drop=True, inplace=True)

In [51]:
# merge into a new data frame with tf-idf features
train_feats2 = pd.concat([train_feats[cols], tfid_df_train], axis=1)
test_feats2 = pd.concat([test_feats[cols], tfid_df_test], axis=1)

### NaiveBayes

In [28]:
# let's check cross validation score of the model
# cv score acts a unbiased estimate of models accuracy on unseen data

mod1 = GaussianNB()
target = train_feats['Is_Response']

In [31]:
## Naive Bayes 1
print(cross_val_score(mod1, train_feats1, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.76311844  0.7745      0.7515      0.765       0.75837919]


In [32]:
## Naive Bayes 2 - tfidf is giving higher CV score
print(cross_val_score(mod1, train_feats2, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.79310345  0.811       0.8035      0.815       0.79789895]


In [36]:
# make our first set of predictions

clf1 = GaussianNB()
clf1.fit(train_feats1, target)

clf2 = GaussianNB()
clf2.fit(train_feats2, target)

GaussianNB(priors=None)

In [41]:
preds1 = clf1.predict(test_feats1)
preds2 = clf2.predict(test_feats2)

In [54]:
def to_labels(x):
    if x == 1:
        return "happy"
    return "not_happy"

In [68]:
sub1 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds1})
sub1['Is_Response'] = sub1['Is_Response'].map(lambda x: to_labels(x))

In [72]:
sub2 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds2})
sub2['Is_Response'] = sub2['Is_Response'].map(lambda x: to_labels(x))

In [83]:
sub1 = sub1[['User_ID', 'Is_Response']]
sub2 = sub2[['User_ID', 'Is_Response']]

In [85]:
## write submission files
sub1.to_csv('submissions/sub1_cv.csv', index=False)
sub2.to_csv('submissions/sub2_tf.csv', index=False)s

### LightGBM - 1

We are prefering lightgbm over xgboost because of its speed. <br />
In this model, we'll use count features for model training.

In [87]:
import lightgbm as lgb

In [88]:
# set the data in format lgb accepts
d_train = lgb.Dataset(train_feats1, label = target)

In [89]:
## set parameters
## you can tune the parameters can try to better score

params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05, 
    'max_depth': 7, 
    'num_leaves': 21, 
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [98]:
lgb_cv = lgb.cv(params, d_train, num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's binary_error: 0.2132 + 0.00456488
[40]	cv_agg's binary_error: 0.195401 + 0.00625882
[60]	cv_agg's binary_error: 0.175601 + 0.00580722
[80]	cv_agg's binary_error: 0.1652 + 0.00589807
[100]	cv_agg's binary_error: 0.1568 + 0.00628195
[120]	cv_agg's binary_error: 0.1505 + 0.00328588
[140]	cv_agg's binary_error: 0.1487 + 0.00399728
[160]	cv_agg's binary_error: 0.147301 + 0.00497347
[180]	cv_agg's binary_error: 0.1445 + 0.00362296
[200]	cv_agg's binary_error: 0.1439 + 0.00429358
[220]	cv_agg's binary_error: 0.1417 + 0.00200147
[240]	cv_agg's binary_error: 0.1418 + 0.0040771
[260]	cv_agg's binary_error: 0.1401 + 0.00373791
[280]	cv_agg's binary_error: 0.1389 + 0.00517039
[300]	cv_agg's binary_error: 0.1376 + 0.00466764
[320]	cv_agg's binary_error: 0.136901 + 0.00507148
[340]	cv_agg's binary_error: 0.1357 + 0.00529898
[360]	cv_agg's binary_error: 0.1363 + 0.00505334
[380]	cv_agg's binary_error: 0.1353 + 0.0044035
[400]	cv_agg's binary_error: 0.1356 + 0.00428458
[420]	cv_agg's bin

In [126]:
## get nround value which hd lowest error
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

In [129]:
## train the model
model = lgb.train(params, d_train, num_boost_round=nround)

In [130]:
## make predictions
preds = model.predict(test_feats1)

In [157]:
# make submission

def to_labels(x):
    if x > 0.66:  # cutoff - you can change it and see if accuracy improves or plot AUC curve. 
        return "happy"
    return "not_happy"

sub3 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub3['Is_Response'] = sub3['Is_Response'].map(lambda x: to_labels(x))
sub3 = sub3[['User_ID','Is_Response']]
sub3.to_csv('submissions/sub3_lgb.csv', index=False) # 0.85518

### LightGBM - 2

In this model, we'll use tf-idf features for model training.

In [140]:
# set data format
d_train = lgb.Dataset(train_feats2, label = target)

In [144]:
# same parameters as above
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05, 
    'max_depth': 5, 
    'num_leaves': 11,
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [145]:
## do cross validation to find nround i.e. at this round (iteration) we can expect lowest error
lgb_cv = lgb.cv(params, d_train, num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's binary_error: 0.226401 + 0.00518217
[40]	cv_agg's binary_error: 0.206602 + 0.00687761
[60]	cv_agg's binary_error: 0.183302 + 0.00791949
[80]	cv_agg's binary_error: 0.169801 + 0.00512283
[100]	cv_agg's binary_error: 0.164301 + 0.00632119
[120]	cv_agg's binary_error: 0.1578 + 0.00507081
[140]	cv_agg's binary_error: 0.1542 + 0.00524522
[160]	cv_agg's binary_error: 0.1516 + 0.00441669
[180]	cv_agg's binary_error: 0.148701 + 0.00512212
[200]	cv_agg's binary_error: 0.1461 + 0.00366096
[220]	cv_agg's binary_error: 0.1443 + 0.00362658
[240]	cv_agg's binary_error: 0.1437 + 0.00471092
[260]	cv_agg's binary_error: 0.143501 + 0.00450031
[280]	cv_agg's binary_error: 0.1405 + 0.00506581
[300]	cv_agg's binary_error: 0.1396 + 0.00578611
[320]	cv_agg's binary_error: 0.137801 + 0.00687571
[340]	cv_agg's binary_error: 0.138701 + 0.00679821
[360]	cv_agg's binary_error: 0.137201 + 0.00839438
[380]	cv_agg's binary_error: 0.137501 + 0.00738158
[400]	cv_agg's binary_error: 0.136401 + 0.00735946


In [146]:
# get nround value
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

In [159]:
# train model
model = lgb.train(params, d_train, num_boost_round=nround)

In [160]:
# make prediction
preds = model.predict(test_feats2)

In [162]:
# make submission

def to_labels(x):
    if x > 0.66:
        return "happy"
    return "not_happy"

sub4 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub4['Is_Response'] = sub4['Is_Response'].map(lambda x: to_labels(x))
sub4 = sub4[['User_ID','Is_Response']]
sub4.to_csv('submissions/sub4_lgb.csv', index=False) # 0.84925

### CatBoost

Catboost is a new package recently launched by Yandex. It is said that it works well when the data has many categorical features. We'll use it on count data and see it our model improves.

In [ ]:
## import library
from catboost import CatBoostClassifier,cv, Pool

In [193]:
## catboost accepts categorical columns as a list of column numbers. In this data, all columns are categorical
cat_cols = [x for x in range(502)] ## 502 == train_feats1.shape[1] 

In [196]:
## set parameters
## you can refer the parameters here: https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_parameters-list-docpage/#python-reference_parameters-list
param = {
    'use_best_model':True,
    'loss_function':'CrossEntropy',
    'eval_metric':'Accuracy',
    'iterations':1000,
    'depth':6,
    'learning_rate':0.03,
    'rsm':0.3,
    'random_seed':2017,
    
    
}

In [197]:
## for doing cross validation, set data in Pool format
my_dt =  Pool(train_feats1, 
           label=target,
           cat_features=cat_cols,
           column_description=None,
           delimiter='\t',
           has_header=None,
           weight=None, 
           baseline=None,
           feature_names=None,
           thread_count=1)

In [198]:
## run cv to get best iteration
ctb_cv = cv(param, my_dt, fold_count=5, random_seed=2017)

Borders generated
0:	0:	0:	0:	0:	1:	1:	1:	1:	1:	2:	2:	2:	2:	2:	3:	3:	3:	3:	3:	4:	4:	4:	4:	4:	5:	5:	5:	5:	5:	6:	6:	6:	6:	6:	7:	7:	7:	7:	7:	8:	8:	8:	8:	8:	9:	9:	9:	9:	9:	10:	10:	10:	10:	10:	11:	11:	11:	11:	11:	12:	12:	12:	12:	12:	13:	13:	13:	13:	13:	14:	14:	14:	14:	14:	15:	15:	15:	15:	15:	16:	16:	16:	16:	16:	17:	17:	17:	17:	17:	18:	18:	18:	18:	18:	19:	19:	19:	19:	19:	20:	20:	20:	20:	20:	21:	21:	21:	21:	21:	22:	22:	22:	22:	22:	23:	23:	23:	23:	23:	24:	24:	24:	24:	24:	25:	25:	25:	25:	25:	26:	26:	26:	26:	26:	27:	27:	27:	27:	27:	28:	28:	28:	28:	28:	29:	29:	29:	29:	29:	30:	30:	30:	30:	30:	31:	31:	31:	31:	31:	32:	32:	32:	32:	32:	33:	33:	33:	33:	33:	34:	34:	34:	34:	34:	35:	35:	35:	35:	35:	36:	36:	36:	36:	36:	37:	37:	37:	37:	37:	38:	38:	38:	38:	38:	39:	39:	39:	39:	39:	40:	40:	40:	40:	40:	41:	41:	41:	41:	41:	42:	42:	42:	42:	42:	43:	43:	43:	43:	43:	44:	44:	44:	44:	44:	45:	45:	45:	45:	45:	46:	46:	46:	46:	46:	47:	47:	47:	47:	47:	48:	48:	48:	48:	48:	49:	49:	49:	49:	49:	50:	50:	50:	50:	50:	51:	51:	51:	

In [205]:
# fetch best round
best_round = ctb_cv['b\'Accuracy\'_test_avg'].index(np.max(ctb_cv['b\'Accuracy\'_test_avg']))

In [206]:
## define the classifer model
model = CatBoostClassifier(iterations=best_round, learning_rate=0.03,rsm = 0.3 ,depth=6, eval_metric='Accuracy', random_seed=2017)

In [207]:
## train model
model.fit(my_dt)

In [208]:
## make predictions
preds = model.predict(test_feats1)

In [222]:
## make submission
sub5 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub5['Is_Response'] = ['happy' if x == 1 else 'not_happy' for x in sub5['Is_Response']]
sub5 = sub5[['User_ID','Is_Response']]
sub5.to_csv('submissions/sub5_cb.csv', index=False)